In [1]:
import sqlite3
import pandas as pd
import numpy as np

In [11]:
query = '''
SELECT *
FROM hop
INNER JOIN npi n1
ON hop.from_npi = n1.npi
INNER JOIN npi n2
ON hop.to_npi = n2.npi
WHERE n1.entity_type_code = 1
    AND n2.location_address_city_name = 'NASHVILLE'
    AND n2.location_address_state_name = 'TN'
    AND n2.entity_type_code = 2
    AND hop.transaction_count >= 50
    AND hop.average_day_wait <= 50;
'''

with sqlite3.connect('../data/hop_db.sqlite') as db: 
    test_df = pd.read_sql(query, db)
    
db.close()

In [12]:
test_df

,from_npi,to_npi,patient_count,transaction_count,average_day_wait,std_day_wait,npi,entity_type_code,organization_name,last_name,...,middle_name,name_prefix,name_suffix,credential,first_line_location_address,second_line_location_address,location_address_city_name,location_address_state_name,location_address_postal_code,taxonomy_code
0,1134210149,1700900040,454,1016,0.000,0.000,1134210149,1.0,None,HORN,...,None,None,None,None,2011 CHURCH ST,SUITE 801,NASHVILLE,TN,37203,207W00000X
1,1811905409,1548450513,59,83,8.193,30.203,1811905409,1.0,None,ALI,...,None,None,None,None,397 WALLACE RD,SUITE 415,NASHVILLE,TN,37211,207RP1001X
2,1568410363,1548450513,48,77,13.312,36.028,1568410363,1.0,None,JAIN,...,None,None,None,None,397 WALLACE RD,SUITE 415,NASHVILLE,TN,37211,207RP1001X
3,1689647489,1548450513,29,51,20.118,46.529,1689647489,1.0,None,PATEL,...,None,None,None,None,397 WALLACE RD,SUITE 415,NASHVILLE,TN,37211,207RP1001X
4,1003876236,1548450513,50,60,24.767,48.130,1003876236,1.0,None,KUHN,...,None,None,None,None,397 WALLACE RD,SUITE 415,NASHVILLE,TN,37211,207RP1001X
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43793,1265884050,1124443403,12,75,0.000,0.000,1265884050,1.0,None,WILLIAMS,...,None,None,None,None,22 CENTURY BLVD STE 220,None,NASHVILLE,TN,37214,207RH0002X
43794,1932600103,1124443403,16,64,0.000,0.000,1932600103,1.0,None,SAMS,...,None,None,None,None,22 CENTURY BLVD STE 220,None,NASHVILLE,TN,37214,207RH0002X
43795,1376080960,1124443403,14,53,0.000,0.000,1376080960,1.0,None,BEARD,...,None,None,None,None,22 CENTURY BLVD STE 220,None,NASHVILLE,TN,37214,207RH0002X
43796,1427046622,1902894330,23,74,3.797,9.352,1427046622,1.0,None,LAYA,...,None,None,None,None,217 W MAPLEWOOD LANE,None,NASHVILLE,TN,37207,3336H0001X


Build a profile of providers referring patients to the major hospitals in Nashville. Are certain specialties more likely to refer to a particular hospital over the others?

In [7]:
query = """
SELECT npi, organization_name,  
FROM npi
LEFT JOIN taxonomy
USING(taxonomy_code)
WHERE entity_type_code = 2
AND location_address_city_name = 'NASHVILLE'
AND classification = 'General Acute Care Hospital'
"""

with sqlite3.connect('../data/hop_db.sqlite') as db: 
    nash_hosp = pd.read_sql(query, db)
    
db.close()

DatabaseError: Execution failed on sql '
SELECT npi, organization_name,  
FROM provider
LEFT JOIN taxonomy
USING(taxonomy_code)
WHERE entity_type_code = 2
AND mailing_address_city_name = 'NASHVILLE'
AND classification = 'General Acute Care Hospital'
': near "FROM": syntax error

In [ ]:
nash_hosp['organization_name'].value_counts().tail(42)

In [ ]:
query = """
WITH nash_hosp AS (
    SELECT npi AS hosp_npi, organization_name AS hospital
    FROM provider
    LEFT JOIN taxonomy
    USING(taxonomy_code)
    WHERE entity_type_code = 2
    AND mailing_address_city_name = 'NASHVILLE'
    AND classification = 'General Acute Care Hospital'
)
SELECT from_npi, display_name, transaction_count, hospital
FROM referrals
INNER JOIN provider
ON from_npi = npi
INNER JOIN nash_hosp
ON to_npi = hosp_npi
LEFT JOIN taxonomy
USING(taxonomy_code)
WHERE transaction_count >= 50
AND average_day_wait > 50
"""

with sqlite3.connect('../data/hop_db.sqlite') as db: 
    nash_hosp_referers = pd.read_sql(query, db)
    
db.close()

In [ ]:
(
    nash_hosp_referers
    .groupby(['display_name', 'hospital'])['transaction_count']
    .sum()
    .to_frame()
    .reset_index()
    .sort_values('transaction_count', ascending = False)
)